In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd

In [0]:
anime=pd.read_csv('animes.csv')

In [0]:
review=pd.read_csv('reviews.csv')

In [0]:
profiles=pd.read_csv('profiles.csv')

In [7]:
anime.shape

(19311, 12)

In [8]:
profiles.shape

(81727, 5)

In [9]:
review.shape

(192112, 7)

In [0]:
anime.drop(['episodes'],axis=1,inplace=True)

In [0]:
anime.drop(['link'],axis=1,inplace=True)

In [0]:
anime.rename(columns={'uid':'anime_id'},inplace=True)

In [0]:
anime.drop_duplicates(subset='anime_id',keep='first',inplace=True)

In [0]:
anime=anime.dropna(axis=0,how="any")

In [15]:
print(anime.isnull().sum())

anime_id      0
title         0
synopsis      0
genre         0
aired         0
members       0
popularity    0
ranked        0
score         0
img_url       0
dtype: int64


In [16]:
anime.shape

(13871, 10)

In [17]:
anime.head()

,anime_id,title,synopsis,genre,aired,members,popularity,ranked,score,img_url
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...


In [0]:
review.drop(['text'],axis=1,inplace=True)

In [0]:
review.drop(['scores'],axis=1,inplace=True)

In [0]:
review.rename(columns={'uid':'review_id','profile':'username','anime_uid':'anime_id'},inplace=True)

In [0]:
review=review.dropna(axis=0,how="any")

In [22]:
print(review.isnull().sum())

review_id    0
username     0
anime_id     0
score        0
link         0
dtype: int64


In [23]:
review.head()

,review_id,username,anime_id,score,link
0,255938,DesolatePsyche,34096,8,https://myanimelist.net/reviews.php?id=255938
1,259117,baekbeans,34599,10,https://myanimelist.net/reviews.php?id=259117
2,253664,skrn,28891,7,https://myanimelist.net/reviews.php?id=253664
3,8254,edgewalker00,2904,9,https://myanimelist.net/reviews.php?id=8254
4,291149,aManOfCulture99,4181,10,https://myanimelist.net/reviews.php?id=291149


In [0]:
user_id=pd.Series([int])
for i in range(len(profiles)):
    user_id[i]=i+1

In [0]:
profiles.insert(1,"user_id",user_id)

In [0]:
profiles.rename(columns={'profile':'username'},inplace=True)

In [27]:
print(profiles.isnull().sum())

username               0
user_id                0
gender             27871
birthday           34920
favorites_anime        0
link                   0
dtype: int64


In [0]:
profiles.drop(['gender'],axis=1,inplace=True)

In [0]:
profiles.drop(['birthday'],axis=1,inplace=True)

In [0]:
profiles.drop(['favorites_anime','link'],axis=1,inplace=True)

In [0]:
profiles.drop_duplicates(subset='username',keep='first',inplace=True)

In [32]:
profiles.head()

,username,user_id
0,DesolatePsyche,1
1,baekbeans,2
2,skrn,3
3,edgewalker00,4
4,aManOfCulture99,5


In [0]:
review_dataset=profiles.merge(review)

In [34]:
review_dataset.head()

,username,user_id,review_id,anime_id,score,link
0,DesolatePsyche,1,255938,34096,8,https://myanimelist.net/reviews.php?id=255938
1,DesolatePsyche,1,255793,12403,6,https://myanimelist.net/reviews.php?id=255793
2,DesolatePsyche,1,298207,35073,6,https://myanimelist.net/reviews.php?id=298207
3,DesolatePsyche,1,255795,23225,7,https://myanimelist.net/reviews.php?id=255795
4,DesolatePsyche,1,291256,2035,8,https://myanimelist.net/reviews.php?id=291256


review_dataset

In [35]:
review_dataset

,username,user_id,review_id,anime_id,score,link
0,DesolatePsyche,1,255938,34096,8,https://myanimelist.net/reviews.php?id=255938
1,DesolatePsyche,1,255793,12403,6,https://myanimelist.net/reviews.php?id=255793
2,DesolatePsyche,1,298207,35073,6,https://myanimelist.net/reviews.php?id=298207
3,DesolatePsyche,1,255795,23225,7,https://myanimelist.net/reviews.php?id=255795
4,DesolatePsyche,1,291256,2035,8,https://myanimelist.net/reviews.php?id=291256
...,...,...,...,...,...,...
192107,daniel1302,77657,320265,2593,8,https://myanimelist.net/reviews.php?id=320265
192108,bridgesams,77658,304201,2593,9,https://myanimelist.net/reviews.php?id=304201
192109,Officer_Anime,77660,257057,2593,2,https://myanimelist.net/reviews.php?id=257057
192110,Yuez,77661,309073,2593,4,https://myanimelist.net/reviews.php?id=309073


# NAIVE BAYES

In [0]:
naive_dataset=anime[['anime_id','title','synopsis','score']]

In [37]:
import nltk
nltk.download('stopwords')
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(naive_dataset,test_size=0.5)

In [0]:
def word_split(dataset):
    dataset['synopsis']=dataset['synopsis'].astype(str).str.lower().str.split()
    dataset['synopsis']=dataset['synopsis'].apply(lambda x: [item for item in x if item not in stop_words])
    return dataset

In [0]:
def count_words(dataset):
    count=pd.Series([float])
    count=dataset['synopsis'].str.len()
    dataset.insert(3,'count',count)
    words_count=dataset['synopsis'].apply(pd.value_counts)
    frequency=words_count.div(count,axis=0).fillna(0).round(2)
    return frequency

In [0]:
def naive_bayes(data):
    naive_data=word_split(data)
    frequency=count_words(naive_data)
    Ai=cosine_similarity(frequency).mean(axis=0)
    binary=pd.Series(int)
    binary=(naive_data['score']/8.5).astype(int)
    naive_data.insert(5,'binary_rat',binary)
    one=(len(naive_data[naive_data.binary_rat==1]))/len(naive_data)
    Ai_one=(naive_data.binary_rat)/len(naive_data[naive_data.binary_rat==1])
    pred=pd.Series(float)
    pred=(Ai_one*one/Ai)
    naive_data.insert(6,'prediction',pred)
    recommend=naive_data.sort_values(by=['prediction'],ascending=False).head()
    naive_data['score']=naive_data['score']/100
    rmse=np.sqrt((naive_data.score-(naive_data.prediction)**2).mean())
    print('RMSE',rmse)
    print(recommend[['anime_id','title','prediction']])

In [42]:
recommendation=naive_bayes(train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


RMSE 0.25246749975337646
       anime_id                                      title  prediction
12771     35151  Quiz de Manabu Pinocchio no Koutsuu Ansen    0.066705
10805     39405                Kaitei-koku no Koutsu Anzen    0.018973
725       32983                      Natsume Yuujinchou Go    0.012595
722        5300                    Zoku Natsume Yuujinchou    0.012521
738       31757            Kizumonogatari II: Nekketsu-hen    0.011970


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [43]:
recommendation=naive_bayes(test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


RMSE 0.2524285798288118
       anime_id                                          title  prediction
5         37510                              Mob Psycho 100 II    0.012573
773        9253                                    Steins;Gate    0.011692
719        4282                 Kara no Kyoukai 5: Mujun Rasen    0.011592
13510     39229  Time Slip 1923: Mori no Miracle Jishin Taiken    0.011436
763         164                                  Mononoke Hime    0.011029


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
